##  1 . Setting up

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Msc_project/neural_graph_collaborative_filtering/NGCF/

/content/gdrive/MyDrive/Msc_project/neural_graph_collaborative_filtering/NGCF


In [3]:
! ls

BPRMF.py  loss_record.png  NMF.py  pretrain   utility
GATCF.py  NGCF.py	   output  README.md  weights


In [ ]:
#! git clone https://github.com/xiangwang1223/neural_graph_collaborative_filtering.git

### Specifying the TensorFlow version

Running `import tensorflow` will import the default version (currently 2.x). You can use 1.x by running a cell with the `tensorflow_version` magic **before** you run `import tensorflow`.



In [4]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
import tensorflow as tf
print(tf.__version__)

1.15.2


If you want to switch TensorFlow versions after import, you **will need to restart your runtime** with 'Runtime' -> 'Restart runtime...' and then specify the version before you import it again.

### Avoid Using ``pip install`` with GPUs and TPUs

We recommend against using ``pip install`` to specify a particular TensorFlow version for both GPU and TPU backends. Colab builds TensorFlow from source to ensure compatibility with our fleet of accelerators. Versions of TensorFlow fetched from PyPI by ``pip`` may suffer from performance problems or may not work at all.

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
#tf.device('/device:GPU:0')

## 2 . Exploratory/Preliminary experiments on a subset of Gowalla dataset

### 2.0 MF baseline

In [ ]:
##the MF, create a baseline —— recall@20: 0.12196 ; NDCG@20: 0.20591; Precision@20: 0.04850
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.001 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 300 --alg_type 'mf'  --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10674071311950684
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization





without pretraining.
Epoch 0 [10.3s]: train==[251467.04688=251467.08691 + 0.00000]
Epoch 1 [10.2s]: train==[238418.60938=238418.62305 + 0.00000]
Epoch 2 [10.0s]: train==[198638.06250=198638.05103 + 0.00000]
Epoch 3 [10.0s]: train==[149119.70312=149119.67615 + 0.00000]
Epoch 4 [10.1s]: train==[109590.78906=109590.81738 + 0.00000]
Epoch 5 [10.1s]: train==[82539.14844=82539.12332 + 0.00000]
Epoch 6 [10.1s]: train==

###2.1 the NGCF (after correcting) on a subset of Gowalla dataset, create a baseline —— recall@20: 0.13430 ; NDCG@20: 0.22296;

In [ ]:
##the NGCF (after correcting), create a baseline —— recall@20: 0.13430 ; NDCG@20: 0.22296; Precision@20: 0.05328
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 700 --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.11084437370300293
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [33.7s]: train==[962740.42139=962739.94092 + 0.47881]
Epoch 1 [26.5s]: train==[604787.39209=604787.05322 + 0.30429]
Epoch 2 [26.6s]: train==[414781.46814=414781.22681 + 0.19907

### Proposal 1: different aggregation rules

#### remove W*ei from original aggregation rule + context-only (uni_norm adj_mat)

In [ ]:
##the NGCF (after correcting) on a subset of Gowalla dataset, with Context-only updater(unidirectionally normalized)
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.001 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 1000 --alg_type 'variant1' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10605216026306152
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
Epoch 0 [30.2s]: train==[963212.31250=963212.24414 + 0.00000]
Epoch 1 [27.7s]: train==[664283.25000=664283.19849 + 0.00000]
Epoch 2 [27.5s]: train==[577766.81250=577766.68677 + 0.00000]
Epoch 3 [27.6s]: train==[510463.75000=510463.67065 + 0.00000]
Epoch 4 [27.7s]: train==[43442

#### remove W*(ei x eu) from original aggregation rule + context-only (uni_norm adj_mat)

In [ ]:
##the NGCF (after correcting) on a subset of Gowalla dataset, with Context-only updater(unidirectionally normalized)
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0005 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 1000 --alg_type 'variant2' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 3.579791784286499
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
Epoch 0 [29.4s]: train==[822330.31250=822330.35254 + 0.00000]
Epoch 1 [25.6s]: train==[466342.93750=466342.69226 + 0.00000]
Epoch 2 [25.8s]: train==[300795.09375=300795.07971 + 0.00000]
Epoch 3 [25.7s]: train==[265969.21875=265969.21655 + 0.00000]
Epoch 4 [25.7s]: train==[236637.

#### W(e_i || e_u)

In [ ]:
## Context-only updater(unidirectionally normalized)
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0005 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 1000 --alg_type 'variant3' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.1053617000579834
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
Epoch 0 [27.4s]: train==[744971.37500=744971.29419 + 0.00000]
Epoch 1 [24.1s]: train==[349746.31250=349746.22119 + 0.00000]
Epoch 2 [26.1s]: train==[231847.92188=231847.72070 + 0.00000]
Epoch 3 [26.4s]: train==[203615.29688=203615.21802 + 0.00000]
Epoch 4 [24.7s]: train==[181797

#### original GATCF-3 layer 

with different lr, best recall@20: 0.0366

In [ ]:
#GATCF-3 layer, using a subset of the original Gowalla dataset, lr = 0.0001
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[128,128,128]" --lr 0.00001 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 50 --alg_type 'gat' --adj_type 'plain' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

In [ ]:
#GATCF-3 layer, using a subset of the original Gowalla dataset, smaller learning rate, lr =0.000001
!python GATCF.py --dataset gowalla --regs "[1e-7]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.000001 --save_flag 1 --pretrain 1 --batch_size 3072 --epoch 80 --alg_type 'gat' --adj_type 'plain' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

##### based on GCN(no_self_loop,no_transform,no_nonlinear)-initalized embeddings 

In [ ]:
#GATCF-3 layer, based on GCN(no_slef_loop,no_transform,no_nonlinear)-initalized embeddings, lr 0.000003 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64]" --lr 0.00003 --save_flag 1 --pretrain -1 --batch_size 10000 --epoch 200 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'plain' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

In [ ]:
#GATCF-3 layer, based on fixed GCN(no_slef_loop,no_transform)-initalized embeddings (临时修改了, fix pretrained embeddings )  
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain -1 --batch_size 4096 --epoch 200 --layers_aggr 'concat' --alg_type 'gat' --adj_type 'plain' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10817980766296387
use the plain adjacency matrix
load the pretrained embeddings.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using pretrained initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
Epoch 0 [124.0s]: train==[29681.15430=29680.72080 + 0.00000]
Epoch 1 [120.8s]: train==[31553.35742=31552.92186 + 0.00000]
Epoch 2 [120.6s]: train==[35731.93750=35731.49152 + 0.00000]
Epoch 3 [120.7s]: train==[125642.65625=125641.96007 + 0.00

#### Modified GAT (make GAT work for implicit feedback-based CF RS)

In [ ]:
## implementation 1: apply softmax on att_mat firstly, then * GCN decay, then softmax again
#(no self_loop,no transformation,non-linear activation)binorm_adjmat,GCN*GAT CF ,on a subset of Gowalla dataset 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0012 --save_flag 1 --pretrain 0 --batch_size 4096 --epoch 100 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'bi_norm' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10759425163269043
use the bidirectionally normalized adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





load the pretrained model parameters from:  weights/gowalla/ngcf_bi_norm_gat_l2/64-64/l0.0012_r1e-05
tcmalloc: large alloc 2685730816 bytes == 0x5638b4468000 @  0x7f4a2644a001 0x7f4a237df54f 0x7f4a2382fb58 0x7f4a23833b17 0x7f4a238d2203 0x563890d9a544 0x563890d9a240 0x563890e0e

In [ ]:
#implementing2, att_mat * GCN decay firstly, then apply softmax, on a subset of Gowalla dataset 
#(no self_loop,no transformation,non-linear activation)binorm_adjmat
#recall=[0.13524], precision=[0.05240], hit=[0.54460], ndcg=[0.21409]
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0013 --save_flag 1 --pretrain 1 --batch_size 2048 --epoch 100 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'bi_norm' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 1.5500049591064453
use the bidirectionally normalized adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





load the pretrained model parameters from:  weights/gowalla/ngcf_bi_norm_gat_l2/64-64/l0.0013_r1e-05
tcmalloc: large alloc 1342865408 bytes == 0x55aab22a6000 @  0x7f8098839001 0x7f8095bce54f 0x7f8095c1eb58 0x7f8095c22b17 0x7f8095cc1203 0x55aa8f64a544 0x55aa8f64a240 0x55aa8f6be6

In [ ]:
#implementing2, with uni_norm_adjmat, on a subset of Gowalla dataset 
#recall=[0.13997], precision=[0.05441], hit=[0.55830], ndcg=[0.22224], MRR=[0.26318]
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.001 --save_flag 1 --pretrain 0 --batch_size 4096 --epoch 1000 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10761237144470215
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
Epoch 0 [58.9s]: train==[969637.62500=969637.57617 + 0.00000]
Epoch 1 [56.5s]: train==[649943.81250=649943.68115 + 0.00000]
Epoch 2 [56.6s]: train==[364798.84375=364798.77026 + 0.00000]
Epoch 3 [56.6s]: train==[244529.96875=244529.96704 + 0.00000]
Epoch 4 [56.7s]: train==[18772

In [ ]:
##GCN*GAT CF, # use inner product to model attention, use inner product to model interaction (failed) on a subset of Gowalla dataset 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0013 --save_flag 1 --pretrain 0 --batch_size 4096 --epoch 300 --intera_func 'inner_prod' --layers_aggr 'avg' --alg_type 'gat2' --adj_type 'bi_norm' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 1.418485164642334
use the bidirectionally normalized adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
tcmalloc: large alloc 10396254208 bytes == 0x5581fb0ea000 @  0x7fc8846e1b6b 0x7fc884701379 0x7fc83af22467 0x7fc83ad0fc4f 0x7fc83abcf019 0x7fc83abd6e6f 0x7fc8443fbaba 0x7fc8443fd11e 0x7fc8443fd30f 0x7fc8443f5b45 0x7fc844374fca 0x7fc844376155 0x7fc84426227b 0x

In [ ]:
##(no self_loop,transformation,non-linear activation)binorm_adjmat,GCN*GAT CF (with softmax) ,based on embeddings generated by GCN
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0015 --save_flag 1 --pretrain -1 --batch_size 4096 --epoch 200 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'bi_norm' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 3.5864880084991455
use the bidirectionally normalized adjacency matrix
load the pretrained embeddings.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using pretrained initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
Epoch 0 [73.9s]: train==[1427.16089=1427.12133 + 0.00000]
Epoch 1 [70.6s]: train==[1292.52917=1292.48919 + 0.00000]
Epoch 2 [70.8s]: train==[1314.01196=1313.97238 + 0.00000]
Epoch 3 [71.0s]: train==[1204.11023=1204.07067 

### Proposal 2: different updating rules 

####  (different) Single-interaction updater 

In [ ]:
##the NGCF (after correcting) on a subset of Gowalla dataset, 
#with Single-interaction updater(concatenation) variant 4   LeakyReLU(W3* (e_u || e_N_u))
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain 0 --batch_size 3072 --epoch 1000 --alg_type 'variant4' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.11071014404296875
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [32.6s]: train==[1007710.58984=1007710.24121 + 0.34196]
Epoch 1 [25.0s]: train==[984754.75879=984754.52441 + 0.25048]
Epoch 2 [25.2s]: train==[690107.10815=690106.92188 + 0.184

In [ ]:
##continue before
#with Single-interaction updater(concatenation) variant 4   LeakyReLU(W3* (e_u || e_N_u))
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain 1 --batch_size 3072 --epoch 1000 --alg_type 'variant4' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10623335838317871
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




load the pretrained model parameters from:  weights/gowalla/ngcf_gcmc_variant4_l3/64-64-64/l0.0001_r1e-05
tcmalloc: large alloc 2014298112 bytes == 0x563bb1094000 @  0x7efd6cecf001 0x7efd6a1a454f 0x7efd6a1

#### Context-only updater 

In [ ]:
## the NGCF (after correcting) on a subset of Gowalla dataset, with Context-only updater (bidirectionally normalized)
#recall=[0.13693], precision=[0.05400], ndcg=[0.22042], MRR=[0.25771]
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain 1 --batch_size 2048 --epoch 500 --alg_type 'ngcf' --adj_type 'bi_norm' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 2.1417315006256104
use the bidirectionally normalized adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




load the pretrained model parameters from:  weights/gowalla/ngcf_bi_norm_ngcf_l3/64-64-64/l0.0001_r1e-05
tcmalloc: large alloc 1342865408 bytes == 0x558bc971e000 @  0x7f2f675ac001 0x7f

In [ ]:
##the NGCF (after correcting) on a subset of Gowalla dataset, with Context-only updater(unidirectionally normalized)
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 1000 --alg_type 'ngcf' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10722708702087402
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [31.7s]: train==[1003770.12500=1003770.18066 + 0.00000]
Epoch 1 [28.8s]: train==[983985.06250=983984.84033 + 0.00000]
Epoch 2 [28.9s]: train==[659128.75000=659128.79468 + 0.000

#### multi-interaction updater

In [ ]:
##the NGCF (after correcting) multi-interaction updater ,on a subset of Gowalla dataset 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.001 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 500 --layers_aggr 'avg' --alg_type 'variant4' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

### Proposal3 —— different layer-aggregaors instead of concatenation

In [ ]:
##the NGCF (after correcting) + average layer aggregator ,on a subset of Gowalla dataset 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.001 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 500 --layers_aggr 'avg' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.1102149486541748
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [31.1s]: train==[841637.25000=841637.37891 + 0.00000]
Epoch 1 [28.3s]: train==[710100.12500=710100.07910 + 0.00000]
Epoch 2 [28.4s]: train==[662189.75000=662189.63208 + 0.00000]

In [ ]:
##the NGCF (after correcting) + weighted average layer aggregator ,on a subset of Gowalla dataset  #Recall: 0.10015
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0009 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 500 --layers_aggr 'weighted_avg' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10851693153381348
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [32.2s]: train==[791222.68750=791222.40161 + 0.00000]
Epoch 1 [29.2s]: train==[615792.00000=615792.04199 + 0.00000]
Epoch 2 [29.4s]: train==[580240.75000=580240.67603 + 0.00000

In [ ]:
##the NGCF (after correcting) + weighted concat layer aggregator ,on a subset of Gowalla dataset 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.00012 --save_flag 1 --pretrain 1 --batch_size 1024 --epoch 300 --layers_aggr 'weighted_concat' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.10705423355102539
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




load the pretrained model parameters from:  weights/gowalla/ngcf_ngcf_ngcf_l3/64-64-64/l0.00012_r1e-05
pretrained model recall=[0.06840, 0.21575], precision=[0.02413, 0.01652], hit=[0.32350, 0.65690],ndcg=

In [ ]:
##the NGCF (after correcting) + last layer aggregator ,on a subset of Gowalla dataset 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0005 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 1000 --layers_aggr 'last' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.11730837821960449
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [35.3s]: train==[733662.70386=733662.46216 + 0.24230]
Epoch 1 [27.9s]: train==[584131.68994=584131.63403 + 0.06320]
Epoch 2 [28.0s]: train==[576372.83423=576372.79077 + 0.05860

### Proposal 4—— neural interaction function

In [ ]:
##the NGCF (after correcting)+ 1 layer neural interaction function  on a subset of Gowalla dataset. recall@20: 0.03851 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.00011 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 300 --intera_func 'neural' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

In [ ]:
##the NGCF (after correcting)+ 2 layer neural interaction function  on a subset of Gowalla dataset, recall@20: 0.08359
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain 1 --batch_size 1024 --epoch 50 --intera_func 'neural' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

In [ ]:
##the NGCF (after correcting)+ 3 layer neural interaction function  on a subset of Gowalla dataset, recall@20: 0.08548
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.00011 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 300 --intera_func 'neural' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 3.5036094188690186
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [29.8s]: train==[225854.04688=225854.01984 + 0.00000]
Epoch 1 [23.7s]: train==[176253.09375=176253.11383 + 0.00000]
Epoch 2 [23.7s]: train==[126669.05469=126669.06500 + 0.00000]

In [ ]:
##the NGCF + 4 layer neural interaction function  on a subset of Gowalla dataset
#recall@20: recall=[0.08917], precision=[0.03457], hit=[0.41400], ndcg=[0.13128], MRR=[0.12870]
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.00011 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 800 --intera_func 'neural' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.11407852172851562
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [37.2s]: train==[864731.55225=864730.93506 + 0.59756]
Epoch 1 [28.7s]: train==[617188.78101=617188.11719 + 0.62417]
Epoch 2 [28.7s]: train==[319059.78601=319059.10706 + 0.64412

In [ ]:
## fusion neural interaction function (3-layer MLP(0.4) + inner product(0.6))  on a subset of Gowalla dataset
#recall=[0.09039], precision=[0.03546	], hit=[0.42700	], ndcg=[0.13565], MRR=[0.11984]
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0005 --save_flag 1 --pretrain 0 --batch_size 2048 --epoch 800 --intera_func 'fusion' --alg_type 'ngcf' --adj_type 'ngcf' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=10000, n_items=40981
n_interactions=458323
n_train=362702, n_test=95621, sparsity=0.00112
already load adj matrix (50981, 50981) 0.11632752418518066
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




without pretraining.
Epoch 0 [36.8s]: train==[660577.66040=660576.97974 + 0.65348]
Epoch 1 [28.6s]: train==[236602.70227=236601.93225 + 0.74874]
Epoch 2 [28.6s]: train==[170428.35925=170427.53076 + 0.79865

##  3 .Verifying those improvemnts on the original Gowalla dataset

### 3.1 the original Github implementation of NGCF

In [ ]:
###the original Github implementation on the original Gowalla dataset
!python NGCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain 1 --batch_size 1024 --epoch 30 --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084
already load adj matrix (70839, 70839) 0.23967933654785156
use the mean adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




load the pretrained model parameters from:  weights/gowalla/ngcf_ngcf_ngcf_l3/64-64-64/l0.0001_r1e-05
pretrained model recall=[0.15576, 0.33514], precision=[0.04776, 0.02114], hit=[0.54133, 0.77674],ndc

### 3.2 Adapting original GAT-2 layers ( doesn't work at all)

In [ ]:
#GATCF-2 layer, using the original Gowalla dataset, doesn't work at all
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.00001 --save_flag 1 --pretrain 1 --batch_size 5120 --epoch 50 --alg_type 'gat' --adj_type 'plain' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1]"

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084
already load adj matrix (70839, 70839) 0.24028754234313965
use the plain adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




load the pretrained model parameters from:  weights/gowalla/ngcf_plain_gat_l2/64-64/l1e-05_r1e-05
pretrained model recall=[0.00548, 0.02069], precision=[0.00156, 0.00114], hit=[0.02720, 0.09063],ndcg=[0

### 3.3 Modified GAT （GCN decay* GAT）

In [ ]:
## implement1 
#2 layers_  the binorm_adj(no self loop,no transform,no nonlinear activation) gcn*GAT_CF ,average of layers embeddings
#recall=[0.15723], precision=[0.04806], hit=[0.53687], ndcg=[0.22608] #batch_size =4096
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0014 --save_flag 1 --pretrain 1 --batch_size 4096 --epoch 100 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'bi_norm' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1]"

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084
already load adj matrix (70839, 70839) 0.25261640548706055
use the bidirectionally normalized adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





load the pretrained model parameters from:  weights/gowalla/ngcf_bi_norm_gat_l2/64-64/l0.0014_r1e-05
tcmalloc: large alloc 2685730816 bytes == 0x56293c262000 @  0x7f6794bd7001 0x7f6791f6c54f 0x7f6791fbcb58 0x7f6791fc0b17 0x7f679205f203 0x5629127d7544 0x5629127d7240 0x5629128

In [ ]:
#implementing2, att_mat * GCN decay firstly, then apply softmax
#(no self_loop,no transformation,non-linear activation) with Max_pooling_normed_adjmat
#recall=[0.15873], precision=[0.04844], hit=[0.54096], ndcg=[0.22793], MRR=[0.25132]
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0013 --save_flag 1 --pretrain 1 --batch_size 2048 --epoch 1000 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084
already load adj matrix (70839, 70839) 2.843208074569702
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





load the pretrained model parameters from:  weights/gowalla/ngcf_gcmc_gat_l2/64-64/l0.0013_r1e-05
tcmalloc: large alloc 2685730816 bytes == 0x558e69f42000 @  0x7f70d0161001 0x7f70cd3b654f 0x7f70cd406b58 0x7f70cd40ab17 0x7f70cd4a9203 0x558e37c25544 0x558e37c25240 0x558e37c99627 0x558e37c939ee 0x558e

In [ ]:
#implementation 3, cancel LeakyRelU on the top of implement 2
#(no self_loop,no transformation,non-linear activation) with uni_norm_adjmat
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0013 --save_flag 1 --pretrain 1 --batch_size 4096 --epoch 1000 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084
already load adj matrix (70839, 70839) 0.23083162307739258
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





load the pretrained model parameters from:  weights/gowalla/ngcf_gcmc_gat_l2/64-64/l0.0013_r1e-05
tcmalloc: large alloc 2685730816 bytes == 0x564e55854000 @  0x7f26cf6ea001 0x7f26cc97f54f 0x7f26cc9cfb58 0x7f26cc9d3b17 0x7f26cca72203 0x564e2409a544 0x564e2409a240 0x564e2410e627 0x564e241089ee 0x56

In [ ]:
#implementation 2, 3-layer
#recall=[0.16304], precision=[0.04991], ndcg=[0.22917], MRR=[0.26440]
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0012 --save_flag 1 --pretrain -1 --batch_size 4096 --epoch 500 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084
already load adj matrix (70839, 70839) 0.233489990234375
use the gcmc adjacency matrix
load the pretrained embeddings.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using pretrained initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
Epoch 0 [327.3s]: train==[33474.76953=33474.52580 + 0.09417]
Epoch 1 [312.6s]: train==[31483.27629=31483.11578 + 0.00842]
Epoch 2 [312.2s]: train==[30459.25967=30459.09035 + 0.01525]
Epoch 3 [312.9s]: train==[29674.11880=29673.93850 + 0.02359

### 3.4 context-only updater

In [ ]:
#context-only updater (with Mean-pooling normized adj_mat), run on the original Gowalla dataset 
!python GATCF.py --dataset gowalla --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0001 --save_flag 1 --pretrain 1 --batch_size 2048 --epoch 1000 --layers_aggr 'concat' --alg_type 'ngcf' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084
already load adj matrix (70839, 70839) 2.957685708999634
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




load the pretrained model parameters from:  weights/gowalla/ngcf_gcmc_ngcf_l3/64-64-64/l0.0001_r1e-05
tcmalloc: large alloc 1342865408 bytes == 0x555a08d80000 @  0x7fdd018b2001 0x7fdcfeb0754f 0x7fdcfeb57b5

## 4 . Verifying those improvements on Yelp2018

#### 4.1 M-GATCF -2 layer

In [ ]:
#implementation 2, 2-layer
!python GATCF.py --dataset yelp2018 --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0014 --save_flag 1 --pretrain 0 --batch_size 4096 --epoch 500 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=31668, n_items=38048
n_interactions=1561406
n_train=1237259, n_test=324147, sparsity=0.00130
already load adj matrix (69716, 69716) 0.3374669551849365
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





without pretraining.
Epoch 0 [500.8s]: train==[1714262.46533=1714262.35889 + 0.07821]
Epoch 1 [486.9s]: train==[700874.80383=700874.63464 + 0.09556]
Epoch 2 [491.8s]: train==[622443.34009=622443.05408 + 0.20989]
Epoch 3 [492.6s]: train==[576446.29089=576445.84534 + 0.33595]
Epoch 4 [494.5s]: trai

In [6]:
#implementation 2, 2-layer
!python GATCF.py --dataset yelp2018 --regs "[1e-5]" --embed_size 64 --layer_size "[64,64]" --lr 0.0014 --save_flag 1 --pretrain 1 --batch_size 5120 --epoch 500 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

n_users=31668, n_items=38048
n_interactions=1561406
n_train=1237259, n_test=324147, sparsity=0.00130
already load adj matrix (69716, 69716) 2.6135191917419434
use the gcmc adjacency matrix

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

using xavier initialization

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





load the pretrained model parameters from:  weights/yelp2018/ngcf_gcmc_gat_l2/64-64/l0.0014_r1e-05
tcmalloc: large alloc 3116892160 bytes == 0x55d6ea1b0000 @  0x7f9844e43001 0x7f984209854f 0x7f98420e8b58 0x7f98420ecb17 0x7f984218b203 0x55d6b0f31544 0x55d6b0f31240 0x55d6b0fa5627 0x55d6b0f9f9ee 0x5

#### 4.2 M-GATCF -3 layer

In [ ]:
#3-layer
!python GATCF.py --dataset yelp2018 --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0014 --save_flag 1 --pretrain -1 --batch_size 5120 --epoch 500 --layers_aggr 'avg' --alg_type 'gat' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"

#### 4.3 Context-only

In [ ]:

!python GATCF.py --dataset yelp2018 --regs "[1e-5]" --embed_size 64 --layer_size "[64,64,64]" --lr 0.0005 --save_flag 1 --pretrain 0 --batch_size 5120 --epoch 500 --layers_aggr 'concat' --alg_type 'gat' --adj_type 'gcmc' --verbose 1 --node_dropout "[0.1]" --mess_dropout "[0.1,0.1,0.1]"